In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2023-09-08T07:27:27.992101-07:00

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 8.14.0

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 5.15.0-1029-nvidia
Machine     : x86_64
Processor   : x86_64
CPU cores   : 256
Architecture: 64bit



In [3]:
%watermark --gpu

GPU Info: 
  GPU 0: NVIDIA A100-SXM4-80GB
  GPU 1: NVIDIA A100-SXM4-80GB
  GPU 2: NVIDIA A100-SXM4-80GB
  GPU 3: NVIDIA A100-SXM4-80GB
  GPU 4: NVIDIA A100-SXM4-80GB
  GPU 5: NVIDIA A100-SXM4-80GB
  GPU 6: NVIDIA A100-SXM4-80GB
  GPU 7: NVIDIA A100-SXM4-80GB



In [4]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
from dask.delayed import delayed
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

import optuna
import gc
import logging

In [5]:
%watermark --iversions

xgboost: 1.7.6
numpy  : 1.24.4
optuna : 3.3.0
logging: 0.5.1.2
dask   : 2023.3.2
pandas : 1.5.3



In [6]:
!nvidia-smi -L

GPU 0: NVIDIA A100-SXM4-80GB (UUID: GPU-cf1d9297-d9c3-6b2a-8afa-e63e4b5abb2d)
GPU 1: NVIDIA A100-SXM4-80GB (UUID: GPU-6d18acdc-2136-40ac-5eeb-724e81476385)
GPU 2: NVIDIA A100-SXM4-80GB (UUID: GPU-a87c7d38-52d2-befc-e2de-3cf90f75d6eb)
GPU 3: NVIDIA A100-SXM4-80GB (UUID: GPU-8e290e7b-dec5-1828-2ec2-a6930ecd4b5b)
GPU 4: NVIDIA A100-SXM4-80GB (UUID: GPU-36f4174f-2697-e22e-3ae5-0d30e814c4f1)
GPU 5: NVIDIA A100-SXM4-80GB (UUID: GPU-d0929bb4-8c8c-8418-70d2-f658013f4b33)
GPU 6: NVIDIA A100-SXM4-80GB (UUID: GPU-78f54132-3566-5a8c-b51b-fb3cc695d648)
GPU 7: NVIDIA A100-SXM4-80GB (UUID: GPU-61b9f63d-cb20-77e8-8b22-40f40fbb8715)


In [7]:
cluster = LocalCUDACluster(n_workers=8)
client = Client(cluster)

2023-09-08 07:27:33,872 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-09-08 07:27:33,872 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-09-08 07:27:33,875 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-09-08 07:27:33,876 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-09-08 07:27:33,882 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-09-08 07:27:33,882 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-09-08 07:27:33,882 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-09-08 07:27:33,882 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-09-08 07:27:33,884 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-09-08 07:27:33,884 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-

In [8]:
!nvidia-smi

Fri Sep  8 07:27:39 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.125.06   Driver Version: 525.125.06   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   34C    P0    70W / 400W |    417MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  On   | 00000000:0F:00.0 Off |                    0 |
| N/A   

In [9]:
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)
 
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

In [10]:
%%time
train_folds = []
val_folds = []
train_ys = []
val_ys = []

for i in range(5):
    print(f'Loading fold {i}')
    train_fold_d = delayed(pd.read_csv)(f'../input/xgtrain_fold_{i}_5X.csv.gz')
    train_fold = dd.from_delayed(train_fold_d)
    
    val_fold_d = delayed(pd.read_csv)(f'../input/xgval_fold_{i}_5X.csv.gz')
    val_fold = dd.from_delayed(val_fold_d)
    
    
    train_y = train_fold['target']
    train_fold = train_fold[train_fold.columns.difference(['target'])]
    
    val_y = val_fold['target']
    val_fold = val_fold[val_fold.columns.difference(['target'])]
    
    train_folds.append(train_fold)
    val_folds.append(val_fold)
    
    train_ys.append(train_y)
    val_ys.append(val_y)

Loading fold 0
Loading fold 1
Loading fold 2
Loading fold 3
Loading fold 4
CPU times: user 1.53 s, sys: 1.51 s, total: 3.04 s
Wall time: 37.9 s


In [11]:
%%time
train = pd.read_csv('../input/train.csv.zip')

target = train['target'].values
target

CPU times: user 1.62 s, sys: 381 ms, total: 2 s
Wall time: 1.81 s


array([0, 0, 0, ..., 0, 0, 0])

In [12]:
train_oof = np.zeros((target.shape[0],))

num_round = 1000

def objective(trial):
        
    params = {
        'objective': trial.suggest_categorical('objective',['binary:logistic']),
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_int('max_depth', 3, 25),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['logloss']),

    }

    kf = KFold(5, shuffle=True, random_state=137)

    for i, (train_index, val_index) in enumerate(kf.split(train,target)):
        dtrain = xgb.dask.DaskDMatrix(client, train_folds[i].values, train_ys[i], enable_categorical=True)
        dval = xgb.dask.DaskDMatrix(client, val_folds[i].values, val_ys[i], enable_categorical=True)
        
        output = xgb.dask.train(client, params, dtrain, num_round)
        booster = output['booster']  # booster is the trained model
        booster.set_param({'predictor': 'gpu_predictor'})
        predictions = xgb.dask.predict(client, booster, dval)
        predictions = predictions.compute()
        train_oof[val_index] = predictions
        del dtrain, dval, output
        gc.collect()
        gc.collect()

    gini = gini_normalized(target, train_oof)
    
    return gini

In [13]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Setup the root logger.
logger.addHandler(logging.FileHandler("optuna_xgb_output_2.log", mode="w"))

optuna.logging.enable_propagation()  # Propagate logs to the root logger.
optuna.logging.disable_default_handler()  # Stop showing logs in sys.stderr.

study = optuna.create_study(storage="sqlite:///xgb_optuna_allstate_2.db", study_name="five_fold_optuna_xgb_2", direction='maximize')

In [14]:
%%time
logger.info("Start optimization.")
study.optimize(objective, n_trials=3)

/tmp/ipykernel_243916/3877596809.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
/tmp/ipykernel_243916/3877596809.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
/tmp/ipykernel_243916/3877596809.py:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
/tmp/ipykernel_243916/3877596809.py:13: FutureWarning: suggest_uniform has been deprecated in v

CPU times: user 1min, sys: 16.8 s, total: 1min 17s
Wall time: 5min 18s


In [15]:
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.head()

,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_objective,params_subsample,params_tree_method,state
0,0,0.279015,0.101644,0.799539,logloss,1.469876,0.020145,6,18,binary:logistic,0.539325,gpu_hist,COMPLETE
1,1,0.206835,7.410294,0.751174,logloss,2.073095,0.088018,7,51,binary:logistic,0.988258,gpu_hist,COMPLETE
2,2,0.276148,2.521110,0.489428,logloss,0.554493,0.004287,16,290,binary:logistic,0.717498,gpu_hist,COMPLETE


In [16]:
%%time
study.optimize(objective, n_trials=5)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_2.csv', index=False)
df.head(8)

/tmp/ipykernel_243916/3877596809.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
/tmp/ipykernel_243916/3877596809.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
/tmp/ipykernel_243916/3877596809.py:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
/tmp/ipykernel_243916/3877596809.py:13: FutureWarning: suggest_uniform has been deprecated in v

CPU times: user 59.4 s, sys: 41.5 s, total: 1min 40s
Wall time: 11min 47s


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_objective,params_subsample,params_tree_method,state
0,0,0.279015,0.101644,0.799539,logloss,1.469876,0.020145,6,18,binary:logistic,0.539325,gpu_hist,COMPLETE
1,1,0.206835,7.410294,0.751174,logloss,2.073095,0.088018,7,51,binary:logistic,0.988258,gpu_hist,COMPLETE
2,2,0.276148,2.521110,0.489428,logloss,0.554493,0.004287,16,290,binary:logistic,0.717498,gpu_hist,COMPLETE
3,3,0.189467,0.022334,0.719523,logloss,0.002333,0.089806,10,194,binary:logistic,0.970214,gpu_hist,COMPLETE
4,4,0.255407,0.230446,0.357887,logloss,0.002049,0.028042,13,197,binary:logistic,0.604751,gpu_hist,COMPLETE
5,5,0.271643,0.949032,0.566390,logloss,3.371826,0.002190,19,156,binary:logistic,0.876918,gpu_hist,COMPLETE
6,6,0.248353,2.229799,0.446748,logloss,0.032399,0.040828,10,167,binary:logistic,0.437375,gpu_hist,COMPLETE
7,7,0.274697,0.010156,0.645108,logloss,0.005530,0.004147,12,38,binary:logistic,0.593388,gpu_hist,COMPLETE


In [17]:
study.best_trial.params

{'objective': 'binary:logistic',
 'tree_method': 'gpu_hist',
 'lambda': 1.4698756145809344,
 'alpha': 0.10164370841935841,
 'colsample_bytree': 0.7995392666832839,
 'subsample': 0.5393248559012488,
 'learning_rate': 0.02014455774890158,
 'max_depth': 6,
 'min_child_weight': 18,
 'eval_metric': 'logloss'}

In [18]:
df.value.max()

0.2790151032993089

In [ ]:
%%time
study.optimize(objective, n_trials=17)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_2.csv', index=False)
df.head(20)

/tmp/ipykernel_243916/3877596809.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
/tmp/ipykernel_243916/3877596809.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
/tmp/ipykernel_243916/3877596809.py:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
/tmp/ipykernel_243916/3877596809.py:13: FutureWarning: suggest_uniform has been deprecated in v

In [ ]:
df.value.max()

In [ ]:
study.best_trial.params

In [ ]:
%%time
study.optimize(objective, n_trials=100)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_2.csv', index=False)
df.head(20)